Data Source: https://www.kaggle.com/snap/amazon-fine-food-reviews 

In [2]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [3]:
import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string
import nltk


In [5]:
con = sqlite3.connect('database.sqlite')

In [6]:
filtered_data = pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score!=3
""",con)

In [8]:
filtered_data.head(3)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...


In [7]:
def partition(x):
    if x<3:
        return 'negative'
    return 'positive'

In [9]:
actual_score = filtered_data['Score']
new_score = actual_score.map(partition)
filtered_data['Score']=new_score

In [10]:
filtered_data.head(3)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...


In [12]:
filtered_data.shape

(525814, 10)

In [13]:
display = pd.read_sql_query("""
SELECT UserId, ProductId, ProfileName, Score, Time, Text, COUNT(*)
FROM Reviews
GROUP BY UserId
HAVING COUNT(*)>1
""",con)

In [15]:
display.head(5)

,UserId,ProductId,ProfileName,Score,Time,Text,COUNT(*)
0,#oc-R115TNMSPFT9I7,B005ZBZLT4,Breyton,2,1331510400,Overall its just OK when considering the price...,2
1,#oc-R11D9D7SHXIJB9,B005HG9ESG,"Louis E. Emory ""hoppy""",5,1342396800,"My wife has recurring extreme muscle spasms, u...",3
2,#oc-R11DNU2NBKQ23Z,B005ZBZLT4,Kim Cieszykowski,1,1348531200,This coffee is horrible and unfortunately not ...,2
3,#oc-R11O5J5ZVQE25C,B005HG9ESG,Penguin Chick,5,1346889600,This will be the bottle that you grab from the...,3
4,#oc-R12KPBODL2B5ZD,B007OSBEV0,Christopher P. Presta,1,1348617600,I didnt like this coffee. Instead of telling y...,2


In [17]:
display.shape

(80668, 7)

In [24]:
display[display["COUNT(*)"]==display["COUNT(*)"] .max()]

,UserId,ProductId,ProfileName,Score,Time,Text,COUNT(*)
57359,A3OXHLG6DIBRW8,B002O3VHXU,"C. F. Hill ""CFH""",5,1282176000,"Green Mountain ""Nantucket Blend"" K-Cups make a...",448


In [19]:
display[display['UserId']=='AZY10LLTJ71NX']

,UserId,ProductId,ProfileName,Score,Time,Text,COUNT(*)
80638,AZY10LLTJ71NX,B001ATMQK2,"undertheshrine ""undertheshrine""",5,1296691200,I bought this 6 pack because for the price tha...,5


In [25]:
sorted_data = filtered_data.sort_values('ProductId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

In [26]:
final = sorted_data.drop_duplicates(subset = {"UserId","ProfileName","Time","Text"}, keep='first', inplace=False) 

In [28]:
final.shape

(364173, 10)

In [34]:
#checking how much data still left
(final['Id'].size*1.0)/(filtered_data['Id'].size*1.0)*100


69.25890143662969

In [35]:
final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]

In [40]:
print(final.shape)
final['Score'].value_counts()

(364171, 10)


positive    307061
negative     57110
Name: Score, dtype: int64

In [42]:
text_value = final['Text'].values

In [46]:
text_value[1000]

"I was really looking forward to these pods based on the reviews.  Starbucks is good, but I prefer bolder taste.... imagine my surprise when I ordered 2 boxes - both were expired! One expired back in 2005 for gosh sakes.  I admit that Amazon agreed to credit me for cost plus part of shipping, but geez, 2 years expired!!!  I'm hoping to find local San Diego area shoppe that carries pods so that I can try something different than starbucks."

In [48]:
import re